In [25]:
from dotenv import load_dotenv
from typing import Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from typing_extensions import TypedDict
from langchain_core.callbacks import StreamingStdOutCallbackHandler
llm = ChatOpenAI(
    api_key="ollama",
    model="llama3.1",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    base_url="http://localhost:11434/v1",
)

In [26]:
class MessageClassifier(BaseModel):
    message_type: Literal["emotional", "logical"] = Field(
        ...,
        description="Classify if the message requires an emotional (therapist) or logical response."
    )

class State(TypedDict):
    messages: Annotated[list, add_messages]
    message_type: str | None

In [27]:
def classify_message(state: State):
    last_message = state["messages"][-1]
    classifier_llm = llm.with_structured_output(MessageClassifier)
    
    result = classifier_llm.invoke([
        {
            "role": "system",
            "content": """Classify the user message as either:
            - 'emotional': if it asks for emotional support, therapy, deals with feelings, or personal problems
            - 'logical': if it asks for facts, information, logical analysis, or practical solutions
            """
        },
        {"role": "user", "content": last_message.content}
    ])
    return {"message_type": result.message_type}

In [28]:
    
def router(state: State):
    message_type = state.get("message_type", "logical")
    if message_type == "emotional":
        return {"next": "therapist"}

    return {"next": "logical"}

In [29]:
def therapist_agent(state: State):
    last_message = state["messages"][-1]

    messages = [
        {"role": "system",
         "content": """You are a compassionate therapist. Focus on the emotional aspects of the user's message.
                        Show empathy, validate their feelings, and help them process their emotions.
                        Ask thoughtful questions to help them explore their feelings more deeply.
                        Avoid giving logical solutions unless explicitly asked."""
         },
        {
            "role": "user",
            "content": last_message.content
        }
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": reply.content}]}

In [30]:
def logical_agent(state: State):
    last_message = state["messages"][-1]

    messages = [
        {"role": "system",
         "content": """You are a purely logical assistant. Focus only on facts and information.
            Provide clear, concise answers based on logic and evidence.
            Do not address emotions or provide emotional support.
            Be direct and straightforward in your responses."""
         },
        {
            "role": "user",
            "content": last_message.content
        }
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": reply.content}]}

In [31]:
graph_builder = StateGraph(State)

graph_builder.add_node("classifier", classify_message)
graph_builder.add_node("router", router)
graph_builder.add_node("therapist", therapist_agent)
graph_builder.add_node("logical", logical_agent)

graph_builder.add_edge(START, "classifier")
graph_builder.add_edge("classifier", "router")

graph_builder.add_conditional_edges(
    "router",
    lambda state: state.get("next"),
    {"therapist": "therapist", "logical": "logical"}
)

graph_builder.add_edge("therapist", END)
graph_builder.add_edge("logical", END)

graph = graph_builder.compile()

In [32]:
def run_chatbot():
    state = {"messages": [], "message_type": None}

    while True:
        user_input = input("Message: ")
        if user_input == "exit":
            print("Bye")
            break

        state["messages"] = state.get("messages", []) + [
            {"role": "user", "content": user_input}
        ]

        state = graph.invoke(state)

        if state.get("messages") and len(state["messages"]) > 0:
            last_message = state["messages"][-1]
            print(f"Assistant: {last_message.content}")


if __name__ == "__main__":
    run_chatbot()

Message:  hey im really sad right now


{ "message_type": "emotional" }It takes a lot of courage to acknowledge and express sadness. What's currently making you feel this way? Is there something specific that's been weighing on your mind or heart lately?Assistant: It takes a lot of courage to acknowledge and express sadness. What's currently making you feel this way? Is there something specific that's been weighing on your mind or heart lately?


Message:  i need some hard solid advice about buying a house


{"message_type":"logical"}Here are some practical and evidence-based considerations to keep in mind when buying a house:

1. **Determine your affordability**: Calculate the maximum home price you can afford based on your income, debts, credit score, and other financial obligations.
2. **Get pre-approved for a mortgage**: Contact a lender or financial institution to verify what loan options are available to you and how much you may be able to borrow.
3. **Consider costs beyond the purchase price**:
	* Closing costs (2-5% of the home's value)
	* Property taxes
	* Homeowners insurance
	* Maintenance and repair costs
4. **Assess local market conditions**: Research the housing market in your desired location, including:
	* Median home prices
	* Average days on market
	* Local economic trends
5. **Evaluating a property**:
	* Inspect the property for potential issues (e.g., pests, structural damage)
	* Consider factors like proximity to amenities, public transportation, and schools
6. **Prior

Message:  exit


Bye
